In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

Using TensorFlow backend.


There are multiple ways of loading data depending on where you load it from. Just run the relevant code block below (or add your own)

Note: I am not sure about  order of psi and phi. 
index => 0,1,2,3,...
pdb => unique name of protein
pdb_aa => amino acid sequence over 21 chars
q8 => sequence over eight chars
dist_matrix =>  ouput 1

FOR PART II
outputs are psi and phi

In [16]:
def load_pickle(datafile):
    # load train data for a specific fold
    with open(datafile, 'rb') as f:
        data =  pickle.load(f)

        
    return data

In [17]:
def produce_test_output(testfile, save_path):
    test_input = pd.read_csv(testfile, header=None)
    protein_names = np.array(test_input.iloc[:,1])
    protein_len = np.array(test_input.iloc[:,2])

    # concatenate all output to one-dimensional
    all_data = []
    all_names = []
    for i, pname in enumerate(protein_names):
        dist_flat = dist[i].ravel()
        array = np.concatenate([dist_flat, psi[i], phi[i]])
        all_data.append(array)

        length = protein_len[i]
        dist_names = ["{}_d_{}_{}".format(pname, i + 1, j + 1) for i in range(length) for
                j in range(length)]

        psi_names = ["{}_psi_{}".format(pname, i + 1) for i in range(length)]
        phi_names = ["{}_phi_{}".format(pname, i + 1) for i in range(length)]
        row_names = np.array(dist_names + psi_names + phi_names)
        all_names.append(row_names)

    all_data = np.concatenate(all_data)
    all_names = np.concatenate(all_names)
    output = {"Id": all_names, "Predicted": all_data}
    output = pd.DataFrame(output)
    output.to_csv(save_path, index=False)

In [60]:
indices, pdbs, length_aas, pdb_aas, q8s, dcalphas, psis, phis, msas = load_pickle("../data/train_fold_3.pkl")

In [56]:
#get the max length of some input sequence
max_length = max(length_aas)

Data Preprocessing

In [71]:
from keras.preprocessing import sequence

In [72]:
from sklearn.preprocessing import OneHotEncoder

In [110]:
#find all amino acids
a = set(pdb_aas[0])
for i in range(len(pdb_aas)):
    a = a.union(pdb_aas[i])

In [111]:
#adding U and O from the set based on wikipedia article (I cannot find 21st amino acid) 
a.add('U')
a.add('O')
#length of a is 22

In [115]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

In [125]:
words = a

In [126]:
vocab_size = len(words)

In [128]:
text = " ".join(list(pdb_aas[2]))

In [130]:
result = one_hot(text, round(vocab_size))

In [134]:
input_train = sequence.pad_sequences(input_train, maxlen=max_length)

NameError: name 'input_train' is not defined

In [133]:
result

[21,
 14,
 14,
 19,
 17,
 16,
 17,
 19,
 19,
 1,
 16,
 16,
 8,
 19,
 16,
 16,
 20,
 14,
 16,
 14,
 15,
 7,
 4,
 6,
 16,
 19,
 10,
 16,
 19,
 7,
 4,
 17,
 17,
 16,
 16,
 16,
 8,
 15,
 16,
 10,
 16,
 17,
 17,
 16,
 16,
 17,
 17,
 6,
 16,
 16,
 6,
 16,
 14,
 1,
 21,
 16,
 19,
 7,
 16,
 1,
 8,
 7,
 14,
 8,
 14,
 7,
 21,
 7,
 6,
 20,
 15,
 16,
 17,
 14,
 6,
 16,
 4,
 14,
 15,
 4,
 1,
 1,
 15,
 19,
 19,
 7,
 4,
 8,
 7,
 15,
 14,
 14,
 17,
 7,
 8,
 1,
 6,
 1,
 1,
 8,
 16,
 1,
 4,
 1,
 7,
 16,
 17,
 8]

In [132]:
pdb_aas[2]

'MAAGELEGGKPLSGLLNALAQDTFHGYPGITEELLRSQLYPEVPPEEFRPFLAKMRGILKSIASADMDFNQLEAFLTAQTKKQGGITSDQAAVISKFWKSHKTKIRES'